### Cold emails for Job hunt
This script was designed to send emails to a bunch of email addresses individually, all at once. Has the functionality to customise emails based on companies. It has placeholder for names too. I have 2 versions. 1. OG - no formatting, simple character string email. 2. HTML version that retains formatting and hyperlinks. Note: Generated using ChatGPT

### Pre-recquisites
**Email Access**
1. Enable 2-Step Verification on your Google Account:
- Go to: https://myaccount.google.com/security
- Under “Signing in to Google”, turn on 2-Step Verification.

2. Create an App Password:
- Go to: https://myaccount.google.com/apppasswords
- Select Mail and "Other" (or your device name)
- Google will generate a 16-character password. Use this password in your Python script (instead of your actual Gmail password)

**I/P file format**
- Has 4 columns: FirstName, LastName, Company, Email
- Email: HTML format to retain formatting like links, bold, etc

In [ ]:
# pip install pandas openpyxl

## OG email sender

In [ ]:
import pandas as pd
import smtplib
from getpass import getpass
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

# Prompt for email and password
sender_email = input("Enter your email address: ")
sender_password = getpass("Enter your email password or app password: ")

# Load Excel file
df = pd.read_excel("Dummy.xlsx")  # Replace with actual filename

# Filter Walmart contacts
walmart_df = df[df['Company'].str.lower() == 'demo']

# Email content
subject = "Excited to Connect!"
body_template = """
Hi {first} {last},

I hope you're doing well! Just wanted to reach out and connect with you regarding a potential opportunity. 
Looking forward to staying in touch.

Best regards,  
Twinkle
"""

# SMTP setup (Gmail example — change if using another provider)
smtp_server = "smtp.gmail.com"
smtp_port = 587

# Start connection
server = smtplib.SMTP(smtp_server, smtp_port)
server.starttls()
server.login(sender_email, sender_password)

# Send personalized emails
for index, row in walmart_df.iterrows():
    receiver_email = row['Email']
    first = row['FirstName']
    last = row['LastName']

    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = receiver_email
    msg['Subject'] = subject

    body = body_template.format(first=first, last=last)
    msg.attach(MIMEText(body, 'plain'))

    try:
        server.sendmail(sender_email, receiver_email, msg.as_string())
        print(f"✅ Sent to {receiver_email}")
        df.at[index, 'Sent'] = 'Yes'
    except Exception as e:
        print(f"❌ Failed to send to {receiver_email}: {e}")
        df.at[index, 'Sent'] = 'Failed'

server.quit()

# Save updated Excel
df.to_excel("updated_output.xlsx", index=False)
print("Updated file saved as 'updated_output.xlsx'")


### Formatted Email

In [ ]:
import pandas as pd
import smtplib
from getpass import getpass
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.application import MIMEApplication
import os

# Prompt for sender credentials
sender_email = input("Enter your email address: ")
sender_password = getpass("Enter your email password or app password: ")

# Load Excel file
df = pd.read_excel("enter_input_email_list.xlsx") 

# Filter Walmart recipients
sub_df = df[df['Company'].str.lower() == 'enter_company_name'][2:]

# Resume file
resume_path = "enter_resume_path.pdf"  

# Email subject
subject = "Excited to Connect!"

# HTML body template
html_body_template = """
<html>
  <body>
    <p>Hi {first},</p>

    <p>Hope you are doing well.</p>

    <p>My name is Twinkle and I recently applied for the <b>Senior Business Analyst</b> position that I believe I am a great fit for.</p>

    <p><b><i>Background</i></b><br>
    I recently completed my MS in Business Analytics from The University of Texas at Austin, where I gained hands-on experience with cutting-edge tools and techniques in analytics. Before that, I spent nearly <b>five years leading data science and analytics initiatives</b>, including managing a team of four.
    </p>

    <p><b><i>Why me?</i></b><br>
    What draws me to the org is the scale and impact of the work.<br>
    I'm a naturally curious person. I come up with meaningful insights and analytical solutions for driving measurable business outcomes.<br>
    </p>

    <p>I would be grateful if you could review my resume and consider me for this role.</p>

    <p>Thank you for your time and consideration,<br><br>

    <b>Twinkle Panda</b><br>
    MSBA Candidate, McCombs School of Business<br>
    University of Texas at Austin |
    <a href="https://github.com/p-twinkle">GitHub</a> |
    <a href="https://www.linkedin.com/in/twinklepanda/">LinkedIn</a> |
    <a href="https://twinklepanda.com/">Portfolio</a>
    </p>
  </body>
</html>
"""

# SMTP setup (Gmail)
smtp_server = "smtp.gmail.com"
smtp_port = 587
server = smtplib.SMTP(smtp_server, smtp_port)
server.starttls()
server.login(sender_email, sender_password)

# Loop through Walmart recipients
for index, row in sub_df.iterrows():
    receiver_email = row['Email']
    first = row['FirstName']

    msg = MIMEMultipart("mixed")
    msg['From'] = sender_email
    msg['To'] = receiver_email
    msg['Subject'] = subject

    # Attach HTML body
    html_body = html_body_template.format(first=first)
    msg.attach(MIMEText(html_body, 'html'))

    # Attach resume
    try:
        with open(resume_path, 'rb') as resume_file:
            part = MIMEApplication(resume_file.read(), Name=os.path.basename(resume_path))
        part['Content-Disposition'] = f'attachment; filename="{os.path.basename(resume_path)}"'
        msg.attach(part)
    except Exception as e:
        print(f"⚠️ Could not attach resume: {e}")
        df.at[index, 'Sent'] = 'Attachment Failed'
        continue

    # Send the email
    try:
        server.sendmail(sender_email, receiver_email, msg.as_string())
        print(f"✅ Sent to {receiver_email}")
        df.at[index, 'Sent'] = 'Yes'
    except Exception as e:
        print(f"❌ Failed to send to {receiver_email}: {e}")
        df.at[index, 'Sent'] = 'Failed'

server.quit()

# Save updated Excel
df.to_excel("updated_output.xlsx", index=False)
print("📄 Updated Excel saved as 'updated_output.xlsx'")

In [ ]:
# END